In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/컴퓨터 비전/computer-vision
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from dataloader.dataset import CustomDataset
from model.unet import UNet
from tool.encode import rle_encode
from tool.transform import fisheye
import albumentations as A
from PIL import Image
from albumentations.pytorch import ToTensorV2

SAVE_ROOT = "/content/drive/MyDrive/컴퓨터 비전/computer-vision/model/checkpoint"
DATA_ROOT = "/content/drive/MyDrive/컴퓨터 비전/data"

def set_dataloader(csv_file):
    def set_tta():
        transform = A.Compose(
          [
              fisheye([-0, 1, 0, 0]),
              A.CenterCrop(550, 1050)
              A.Resize(224, 224),
              A.Normalize(),
              ToTensorV2()
          ])
        return transform
    test_dataset = CustomDataset(data_root = DATA_ROOT, csv_file=csv_file, transform=set_tta(), infer=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=4)
    return test_dataloader

def set_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNet().to(device)
    return model, device

def train():
  model, device = set_model()
  train_dataloader = set_dataloader('train_source.csv')
  model_name = "unet"
  val_every = 5
  max_epoch = 20

  # loss function과 optimizer 정의
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

  # Training loop
  for epoch in range(max_epoch):
      model.train()
      epoch_loss = 0
      for images, masks in tqdm(train_dataloader):
          images = images.float().to(device)
          masks = masks.long().to(device)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, masks.squeeze(1))
          loss.backward()
          optimizer.step()

          epoch_loss += loss.item()

      loss = epoch_loss/len(train_dataloader)
      print(f'Epoch {epoch+1}, Loss: {loss}')

      # Validation loop
      if epoch==0 or (epoch+1)%val_every==0:

          torch.save(model, os.path.join(SAVE_ROOT, model_name+f"_epoch{epoch+1}.pt"))

train()

/content/drive/MyDrive/컴퓨터 비전/computer-vision


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 0/138 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 138/138 [10:25<00:00,  4.54s/it]


Epoch 1, Loss: 0.7680888681308083


100%|██████████| 138/138 [06:49<00:00,  2.97s/it]


Epoch 2, Loss: 0.3242853717959445


...

100%|██████████| 138/138 [06:53<00:00,  2.99s/it]


Epoch 5, Loss: 0.21628202033647592


100%|██████████| 138/138 [06:44<00:00,  2.93s/it]


Epoch 6, Loss: 0.18715577691361523


 59%|█████▊    | 81/138 [04:10<04:30,  4.74s/it]